### ConversationSummaryMemory

`ConversationSummaryMemory`는 시간 경과에 따른 대화의 요약 을 생성합니다. 이는 시간 경과에 따른 대화의 정보를 압축하는 데 유용할 수 있습니다.

대화 요약 메모리는 대화가 진행되는 동안 대화를 요약하고 현재 요약을 메모리에 저장 합니다.

그런 다음 이 메모리를 사용하여 지금까지의 대화 요약을 프롬프트/체인에 삽입할 수 있습니다.

이 메모리는 과거 메시지 기록을 프롬프트에 그대로 보관하면 토큰을 너무 많이 차지할 수 있는 긴 대화에 가장 유용합니다.

`ConversationSummaryMemory` 를 생성합니다.

- 디테일에 있어서 `ConversationSummaryMemory`가 유리할 수 있다. 우리가 원하는 정보가 지식그래프나 Entity에 없으면 제대로 retriever를 하지 못하는 경우가 있다.
- 요약본은 prompt가 영어이므로 출력이 영어일 수 있다. 하지만 한국어로 질문해도 잘 된다.
- `ConversationSummaryBufferMemory` 의 경우에는 일정 토큰까지는 원본 그대로 가지고 있다가, 일정 토큰이 넘어가면 요약본을 가지고 있는다. 이전 대화내용을 그대로 살려서 요약이나 번역하는데 훨씬 용이하다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

memory = ConversationSummaryMemory(
    llm=llm, return_messages=True
)

In [3]:
memory.save_context(
    inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
    outputs={
        "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
    },
)
memory.save_context(
    inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
    outputs={
        "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "여행자 보험은 포함되어 있나요?"},
    outputs={
        "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
    },
)
memory.save_context(
    inputs={
        "human": "항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?"
    },
    outputs={
        "ai": "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
    },
)
memory.save_context(
    inputs={"human": "패키지에 포함된 호텔의 등급은 어떻게 되나요?"},
    outputs={
        "ai": "이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다."
    },
)
memory.save_context(
    inputs={"human": "식사 옵션에 대해 더 자세히 알려주실 수 있나요?"},
    outputs={
        "ai": "이 여행 패키지는 매일 아침 호텔에서 제공되는 조식을 포함하고 있습니다. 점심과 저녁 식사는 포함되어 있지 않아, 여행자가 자유롭게 현지의 다양한 음식을 경험할 수 있는 기회를 제공합니다. 또한, 각 도시별로 추천 식당 리스트를 제공하여 현지의 맛을 최대한 즐길 수 있도록 도와드립니다."
    },
)
memory.save_context(
    inputs={"human": "패키지 예약 시 예약금은 얼마인가요? 취소 정책은 어떻게 되나요?"},
    outputs={
        "ai": "패키지 예약 시 500유로의 예약금이 필요합니다. 취소 정책은 예약일로부터 30일 전까지는 전액 환불이 가능하며, 이후 취소 시에는 예약금이 환불되지 않습니다. 여행 시작일로부터 14일 전 취소 시 50%의 비용이 청구되며, 그 이후는 전액 비용이 청구됩니다."
    },
)

In [4]:
# 저장된 메모리 확인
print(memory.load_memory_variables({})["history"])

[SystemMessage(content='The human asks about the price of a European travel package. The AI responds that the basic price for a 14-night, 15-day package is 3,500 euros, which includes airfare, hotel accommodations, and entrance fees to designated tourist sites, with additional costs depending on selected optional tours or personal expenses. The human then inquires about the main tourist attractions included in the trip, and the AI lists famous sites such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and the Rhine Falls in Zurich, highlighting that travelers will experience the iconic landmarks of each city. The human further asks if traveler insurance is included, and the AI confirms that basic traveler insurance is provided, covering medical expenses and emergency support, with options for additional coverage available. The human then asks if it is possible to upgrade to business class for the flights and inquires about the cost. The AI confirms 

### ConversationSummaryBufferMemory
ConversationSummaryBufferMemory 는 두 가지 아이디어를 결합한 것.

최근 대화내용의 버퍼를 메모리에 유지하되, 이전 대화내용을 완전히 플러시(flush)하지 않고 요약으로 컴파일하여 두 가지를 모두 사용합니다.

대화내용을 플러시할 시기를 결정하기 위해 상호작용의 개수가 아닌 토큰 길이 를 사용합니다.

- 즉, 일정 메모리를 유지하되, 이 일정 수준의 메모리가 넘어가면, 요약해서 관리ㅡㄹㄹ 한다.

In [5]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200, #요약의 기준이 되는 토큰 길이를 설정
    return_messages=True
)

In [6]:
memory.save_context(
    inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
    outputs={
        "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
    },
)

In [7]:
# 메모리에 저장된 대화내용 확인 - max token을 넘어가지 않았으므로 원본 그대로 가지고 있다.
memory.load_memory_variables({})["history"]

[HumanMessage(content='유럽 여행 패키지의 가격은 얼마인가요?'),
 AIMessage(content='유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다.')]

In [8]:
memory.save_context(
    inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
    outputs={
        "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "여행자 보험은 포함되어 있나요?"},
    outputs={
        "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
    },
)
memory.save_context(
    inputs={
        "human": "항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?"
    },
    outputs={
        "ai": "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
    },
)
memory.save_context(
    inputs={"human": "패키지에 포함된 호텔의 등급은 어떻게 되나요?"},
    outputs={
        "ai": "이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다."
    },
)

In [9]:
# 메모리에 저장된 대화내용 확인
memory.load_memory_variables({})["history"]

[SystemMessage(content='The human asks about the price of a European travel package. The AI responds that the basic price for a 14-night, 15-day package is 3,500 euros, which includes airfare, hotel accommodations, and entrance fees to designated tourist sites, with additional costs depending on selected optional tours or personal expenses. The human then inquires about the main tourist attractions included in the trip, and the AI lists famous sites such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and the Rhine Falls in Zurich, highlighting that travelers will experience the iconic landmarks of each city. The human further asks if traveler insurance is included, and the AI confirms that basic traveler insurance is provided, covering medical expenses and emergency support, with options for additional coverage available.'),
 HumanMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?'),
 AIMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다

In [10]:
print(memory.load_memory_variables({})["history"])

[SystemMessage(content='The human asks about the price of a European travel package. The AI responds that the basic price for a 14-night, 15-day package is 3,500 euros, which includes airfare, hotel accommodations, and entrance fees to designated tourist sites, with additional costs depending on selected optional tours or personal expenses. The human then inquires about the main tourist attractions included in the trip, and the AI lists famous sites such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and the Rhine Falls in Zurich, highlighting that travelers will experience the iconic landmarks of each city. The human further asks if traveler insurance is included, and the AI confirms that basic traveler insurance is provided, covering medical expenses and emergency support, with options for additional coverage available.'), HumanMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?'), AIMessage(content='항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 